In [147]:
# AV submission 2
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder  
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier#export graph_viz
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder  
from sklearn.metrics import mean_squared_error
from statistics import mode

In [88]:
dataframe=pd.read_csv("loan_train.csv")
dataframe.head(5)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [89]:
dataframe.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [90]:
def datafil(dataframe):
    dataframe['ApplicantIncome']=dataframe['ApplicantIncome']/1000
    dataframe['CoapplicantIncome']=dataframe['CoapplicantIncome']/1000
    dataframe['Income']=dataframe['ApplicantIncome']+dataframe['CoapplicantIncome']
    dataframe['Loan_Amount_Term']=dataframe['Loan_Amount_Term']/12
    dataframe['Gender'].fillna(dataframe['Gender'].mode()[0],inplace=True)
    dataframe['Married'].fillna(dataframe['Married'].mode()[0],inplace=True)
    dataframe['Education'].fillna(dataframe['Education'].mode()[0],inplace=True)
    dataframe['Self_Employed'].fillna(dataframe['Self_Employed'].mode()[0],inplace=True)
    dataframe['Loan_Amount_Term'].fillna(dataframe['Loan_Amount_Term'].mode()[0],inplace=True)
    dataframe['Credit_History'].fillna(dataframe['Credit_History'].mode()[0],inplace=True)
    dataframe['Property_Area'].fillna(dataframe['Property_Area'].mode()[0],inplace=True)
    dataframe['Dependents'].fillna(dataframe['Dependents'].mode()[0],inplace=True)
    dataframe['LoanAmount'].fillna(dataframe['LoanAmount'].median(),inplace=True)
    labelE=LabelEncoder()
    categorical=['Gender','Married','Education','Self_Employed','Property_Area','Loan_Status']
    for x in categorical:
        dataframe[x]=labelE.fit_transform(dataframe[x])
    dataframe['Dependents'].replace(["0","1","2","3+"],[0,1,2,3],inplace=True)
    dataframe.head()

In [91]:
def datafilfortest(dataframe):
    dataframe['ApplicantIncome']=dataframe['ApplicantIncome']/1000
    dataframe['CoapplicantIncome']=dataframe['CoapplicantIncome']/1000
    dataframe['Income']=dataframe['ApplicantIncome']+dataframe['CoapplicantIncome']
    dataframe['Loan_Amount_Term']=dataframe['Loan_Amount_Term']/12
    dataframe['Gender'].fillna(dataframe['Gender'].mode()[0],inplace=True)
    dataframe['Married'].fillna(dataframe['Married'].mode()[0],inplace=True)
    dataframe['Education'].fillna(dataframe['Education'].mode()[0],inplace=True)
    dataframe['Self_Employed'].fillna(dataframe['Self_Employed'].mode()[0],inplace=True)
    dataframe['Loan_Amount_Term'].fillna(dataframe['Loan_Amount_Term'].mode()[0],inplace=True)
    dataframe['Credit_History'].fillna(dataframe['Credit_History'].mode()[0],inplace=True)
    dataframe['Property_Area'].fillna(dataframe['Property_Area'].mode()[0],inplace=True)
    dataframe['Dependents'].fillna(dataframe['Dependents'].mode()[0],inplace=True)
    dataframe['LoanAmount'].fillna(dataframe['LoanAmount'].median(),inplace=True)
    labelE=LabelEncoder()
    categorical=['Gender','Married','Education','Self_Employed','Property_Area']
    for x in categorical:
        dataframe[x]=labelE.fit_transform(dataframe[x])
    dataframe['Dependents'].replace(["0","1","2","3+"],[0,1,2,3],inplace=True)
    dataframe.head()

In [92]:
datafil(dataframe)
dataframe.describe()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Income
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,0.817590,0.653094,0.744300,0.218241,0.133550,5.403459,1.621246,145.752443,28.534202,0.855049,1.037459,0.687296,7.024705
std,0.386497,0.476373,1.009623,0.413389,0.340446,6.109042,2.926248,84.107233,5.369052,0.352339,0.787482,0.463973,6.458664
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.150000,0.000000,9.000000,1.000000,0.000000,0.000000,0.000000,1.442000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,2.877500,0.000000,100.250000,30.000000,1.000000,0.000000,0.000000,4.166000
50%,1.000000,1.000000,0.000000,0.000000,0.000000,3.812500,1.188500,128.000000,30.000000,1.000000,1.000000,1.000000,5.416500
75%,1.000000,1.000000,1.000000,0.000000,0.000000,5.795000,2.297250,164.750000,30.000000,1.000000,2.000000,1.000000,7.521750
max,1.000000,1.000000,3.000000,1.000000,1.000000,81.000000,41.667000,700.000000,40.000000,1.000000,2.000000,1.000000,81.000000


In [61]:
predictors=['Gender', 'Married', 'Dependents', 'Education','LoanAmount','Income',
       'Self_Employed', 'Loan_Amount_Term', 'Credit_History', 'Property_Area']
outcome='Loan_Status'
model=RandomForestClassifier(n_estimators=100, min_samples_split=25, max_depth=7, max_features=len(predictors),random_state=100)
model.fit(dataframe[predictors],dataframe[outcome])
feat_importance = pd.Series(model.feature_importances_, index=predictors).sort_values(ascending=False)
print (feat_importance)

Credit_History      0.612489
Income              0.180209
LoanAmount          0.100083
Loan_Amount_Term    0.042572
Property_Area       0.022449
Married             0.015800
Education           0.007969
Dependents          0.007945
Self_Employed       0.007476
Gender              0.003007
dtype: float64


In [116]:
def classification_model(model,data,predictors,outcome):
    model.fit(data[predictors],data[outcome])
    prediction=model.predict(data[predictors])
    accuracy=metrics.accuracy_score(prediction,data[outcome])
    print ("Accuracy : %s" % "{0:.3%}".format(accuracy))
    kth=KFold(n_splits=4)
    kth.get_n_splits(data)
    error=[]
    for train,test in kth.split(data):
        train_predictors=(data[predictors].iloc[train,:])
        train_target=data[outcome].iloc[train]
        model.fit(train_predictors,train_target)
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
    print ("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))
    model.fit(data[predictors],data[outcome])    

In [117]:
predictors=['Credit_History','Income','LoanAmount','Loan_Amount_Term','Property_Area']
#xtrain,xtest,ytrain,ytest=train_test_split(dataframe[predictors],dataframe[outcome],test_size=0.33,random_state=42)

In [161]:
#model=xgb.XGBClassifier()
model1=RandomForestClassifier(n_estimators=100, min_samples_split=25, max_depth=7, max_features=len(predictors),random_state=100)
model2=xgb.XGBClassifier()
model3=LogisticRegression(solver='lbfgs')
model4=GaussianNB()
#model5=MLPClassifier()
model6=SVC()
#model7=KNeighborsClassifier()
#model8=DecisionTreeClassifier()
print("RFC")
classification_model(model1,dataframe,predictors,outcome)
print("XGBoost")
classification_model(model2,dataframe,predictors,outcome)
print("LogisticRegression")
classification_model(model3,dataframe,predictors,outcome)
print("GaussianNB")
classification_model(model4,dataframe,predictors,outcome)
print("svm")
classification_model(model6,dataframe,predictors,outcome)

RFC
Accuracy : 82.899%
Cross-Validation Score : 80.136%
XGBoost
Accuracy : 86.319%
Cross-Validation Score : 80.134%
LogisticRegression
Accuracy : 80.945%
Cross-Validation Score : 80.951%
GaussianNB
Accuracy : 80.619%
Cross-Validation Score : 80.138%
svm
Accuracy : 86.971%
Cross-Validation Score : 67.592%


C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: 

In [164]:
actualtest=pd.read_csv('testfile.csv')

In [165]:
actualtest.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [166]:
datafilfortest(actualtest)
actualtest.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Income
0,LP001015,1,1,0,0,0,5.720,0.000,110.0,30.0,1.0,2,5.720
1,LP001022,1,1,1,0,0,3.076,1.500,126.0,30.0,1.0,2,4.576
2,LP001031,1,1,2,0,0,5.000,1.800,208.0,30.0,1.0,2,6.800
3,LP001035,1,1,2,0,0,2.340,2.546,100.0,30.0,1.0,2,4.886
4,LP001051,1,0,0,1,0,3.276,0.000,78.0,30.0,1.0,2,3.276


In [168]:
pred1=model1.predict(actualtest[predictors])
pred2=model2.predict(actualtest[predictors])
pred3=model3.predict(actualtest[predictors])
pred4=model4.predict(actualtest[predictors])
pred5=model5.predict(actualtest[predictors])
pred6=model6.predict(actualtest[predictors])
pred8=model8.predict(actualtest[predictors])
finalDF=pd.DataFrame(actualtest['Loan_ID'])
final=np.array([])
for i in range(0,len(actualtest)):
    final=np.append(final,mode([pred1[i],pred2[i],pred3[i],pred4[i],pred6[i]]))
finalDF['Loan_Status']=final
finalDF['Loan_Status'].replace( [1, 0],['Y', 'N'], inplace=True)
finalDF['Loan_Status'].value_counts()

Y    306
N     61
Name: Loan_Status, dtype: int64

In [169]:
finalDF.to_csv('LP3_AV.csv',index=False)